**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

This notebook can be run interactively or in command line mode. To run in command line mode, use:
```bash
    jupyter nbconvert --to notebook --execute demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb \
  --ExecutePreprocessor.kernel_name=qmcpy --ExecutePreprocessor.timeout=3600 --inplace
```

Our presentation slides for ParslFest are available at [Figma](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=174-95&t=t3jENVMltXWwdLdb-0).

In [1]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

In [ ]:
import sys
import os
import time

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags
# Set force_compute=False to reuse existing outputs; set True to force re-run
force_compute = True
is_debug = False

# Create output directory if it doesn't exist
print(f"{os.getcwd() = }")
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Clean local only files if force_compute is set
if force_compute:
    !(cd ../../.. && make clean_local_only_files)
    # remove output directory contents
    !rm -fr output/*.* runinfo/

os.getcwd() = '/workspaces/QMCSoftware/demos/talk_paper_demos/parsel_fest_2025'
rm -fr test/booktests/.ipynb_checkpoints/
chmod +x scripts/find_local_only_folders.sh > /dev/null 2>&1
for f in ; do \
	rm -f "$f"; > /dev/null 2>&1; \
done


## 1. Sequential Execution

In [3]:
seq_fname = os.path.join(output_dir, "sequential_time.csv")
if (not os.path.exists(seq_fname)) or force_compute:
    start_time = time.time()
    if is_debug:
        !(cd ../../.. && make booktests_no_docker TESTS="tb_quickstart tb_qmcpy_intro tb_quickstart tb_qmcpy_intro")
    else:
        !(cd ../../.. && make booktests_no_docker )
    sequential_time = time.time() - start_time
    
    # save sequential time to a file in output_dir
    with open(seq_fname, "w") as f:
        f.write(f"{sequential_time:.2f}\n")

rm -fr demos/.ipynb_checkpoints/*checkpoint.ipynb && \
find demos -name '*.ipynb' | while read nb; do \
	base=$(basename "$nb" .ipynb); \
	test_base=$(echo "$base" | sed 's/[-.]/_/g'); \
	if [ "$base" != "parsl_fest_2025" ] && [ "$base" != "01_sequential" ] && [ "$base" != "02_parallel" ] && [ "$base" != "03_visualize_speedup" ] && ! ls test/booktests/tb_"$test_base".py > /dev/null 2>&1; then \
		echo "    Missing test for: $nb -> Expected: test/booktests/tb_$test_base.py"; \
	fi; \
done
Total notebooks:  36
Total test files: 33

Generating missing booktest files...
cd test/booktests/ && python generate_test.py --check-missing
No missing test files found.
Total notebooks:  36
Total test files: 33

Generating missing booktest files...
cd test/booktests/ && python generate_test.py --check-missing
No missing test files found.
rm -fr test/booktests/.ipynb_checkpoints/
chmod +x scripts/find_local_only_folders.sh > /dev/null 2>&1
for f in ; do \
	rm -f "$f"; > /dev/null 2>&1; \
done

Noteboo

In [4]:
# free memory
import gc
gc.collect();